In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from nltk.corpus import stopwords
import ipywidgets as widgets

idx={}
mat=0
doc=[]

In [2]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    global idx
    
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [3]:
def buildMatrix(docs,test):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(test)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in test:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            if cnt[k] not in idx:
                continue
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [4]:
def cmer(text,cmer=3,has_cut=True):
    r""" Given a text and parameter c, return the vector of c-mers associated with the text
    """
    temp=text
    if not has_cut:
        stop_words = set(stopwords.words('english'))
        for t in text:
            t=t.lower()
            x=t.split()
    #     temp=x
        temp=[]
        for i in x:
            if i not in stop_words:
                i=st1.stem(i)
                temp.append(i)
        text=temp
    v=[]
    textLength=len(text)
    p = 0
    while(p + cmer <= textLength):
        s=""
        for i in range(cmer):
            s+=text[p+i]
        v.append(s)
        p += 1
    
    return v

In [5]:
def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )
        
def textsToMatrix(doc, c):
    docs = [cmer(t, c) for t in doc]
    return build_matrix(docs)

In [6]:
newdoc=[]
for word in doc[0]:
    word=st1.stem(word)
    newdoc.append(word)
doc[0]=newdoc
print(doc[0])

IndexError: list index out of range

In [7]:
from nltk.stem.lancaster import LancasterStemmer
st1 = LancasterStemmer()
stop_words = set(stopwords.words('english'))
# read test data and class info
df = pd.read_table('./train.dat',sep='\t',header=None)
texts = df.ix[:,:].values

# doc=pd.read_table('./format.dat',sep='\t',header=None)
# docs=doc.ix[:,:].values
ori_docs=[]
cls=[]
for i in texts:
    cls.append(i[0])
    ori_docs.append(i[1:])
test = pd.read_table('./test.dat',sep='\t',header=None)
test = test.ix[::].values
#   upload data for test label
t_cls = pd.read_table('./format.dat',sep='\t',header=None)
t_cls = t_cls.ix[::].values
test_cls = []
global doc
doc=[]
for i in t_cls:
    test_cls.append(i[0])
for d in ori_docs:
    for t in d:
        x=t.split()    
    result=[]
    for i in x:
        if i not in stop_words:
            i=i.lower()
# stem words to small data
            i=st1.stem(i)
            result.append(i)
    doc.append(result)
# testset=[]
# for te in test:
#     for se in te:
#         p=se.split()
#     re=[]
#     for j in p:
#         if j not in stop_words:
#             re.append(j.lower())
#     testset.append(re)


# mattest=textsToMatrix(testset,c=3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [8]:
def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [9]:
# Expect a single post cmer processed test input array. Like ["atest", "testcase"]
def build_test_input_csr_matrix(test):
    size=len(set(test))
    ind = np.zeros(size, dtype=np.int)
    val = np.zeros(size, dtype=np.double)
    ptr = np.zeros(2, dtype=np.int)
    
    cnt = Counter(test)
    keys = list(k for k,_ in cnt.most_common())
    l = len(keys)
    for j,k in enumerate(keys):
        if k not in idx:
            continue
        ind[j] = idx[k]
        val[j] = cnt[k]
    ptr[1] = ptr[0] + l
    
    mat = csr_matrix((val, ind, ptr), shape=(1, len(idx)), dtype=np.double)
    mat.sort_indices()
    return mat

In [10]:
def findNeighbors(test,doc,k=1,c=1):
#   find test in doc and use index in mat
    id=-1
    for i in range(len(doc)):
        if(np.array_equal(test,doc[i])):
            id=i
            break
#     mat=textsToMatrix(doc,c)
#   control if test is not in training set
    
    if(id==-1):
        test=cmer(test,c,has_cut=False)
        test_matrix=build_test_input_csr_matrix(test)
        csr_l2normalize(test_matrix)
        x=test_matrix
#         for k in range(len(testset)):
#             if(np.array_equal(test,testset[i])):
#                 id = k
#                 test=mattest[id]
    else:
        x = mat[id]
#   claculate cosine similarity
    csr_l2normalize(mat)
    dots = x.dot(mat.T)
    if id != -1:
        dots[0,id] = 1 # invalidate self-similarity
    sims = list(zip(dots.indices, dots.data))
    sims.sort(key=lambda x: x[1], reverse=True)
#     print(sims)
    return [[s[0],s[1]]for s in sims[:k] if s[1] > 0 ]

In [11]:
def classifyclass(test,training=doc,cls=cls,k=10,c=3):
    pred=[0.0,0.0,0.0,0.0,0.0]
    neighbor=findNeighbors(test,training,k,c)
    
#   find neighbors class
    for i in range(len(neighbor)):
        label=cls[neighbor[i][0]]-1
#         print(neighbor[i],cls[neighbor[i]])
        pred[label] = pred[label] + neighbor[i][1]
    maxid=0
    for i in range(5):
        if(pred[maxid] < pred[i]):
            maxid=i
#   use majority vote rule to predict label 
    
    return maxid+1

In [12]:
global mat
mat=textsToMatrix(doc,c=3)

In [ ]:
nn=classifyclass(test[:8],doc,cls,k=5)

In [14]:
print(nn)

3


In [ ]:
csr_info(mat)

In [ ]:
classifyclass(doc[1],doc,cls)

In [ ]:
for C in range(1,7):
    global mat
    mat=textsToMatrix(doc,c=C)
    y_p=[]
    progress=widgets.IntProgress(
        value=0,
        min=0,
        max=len(test),
        step=1,
        description='cmer c=' + str(C),
        bar_style='', # 'success', 'info', 'warning', 'danger' or ''
        orientation='horizontal'
    )
    display(progress)
    for text1 in test:
        result=classifyclass(text1,doc,cls,c=C)
    #     print(result)
        y_p.append(result)
        progress.value+=1
    count=0
#     print(y_p)
#     for i in range(len(y_p)):
#         if(y_p[i]==cls[i]):
#             count +=1
#     print("c=" + str(C) + ", accuracy:" + str(count/len(y_p)))
    with open('test_c_'+str(C)+'.txt', 'w') as f:
        for item in y_p:
            f.write("%s\n" % item)
        print('test_c_'+str(C)+'.txt write sucessful!')

In [16]:
y_pred=[]
for text in test[:10]:
    y_pred.append(classifyclass(text,doc,cls,10,3))
print(y_pred)

[4, 1, 5, 2, 1, 4, 1, 1, 3, 4]


In [ ]:
print(idx['laboratory'])

In [ ]:
count=0
for i in range(len(y_pred)):
    if(y_pred[i]==cls[i]):
        count += 1
print(count/len(y_pred))

In [ ]:
print(len(doc))

In [ ]:
newdoc=[]
for word in doc[0]:
    word=st1.stem(word)
    newdoc.append(word)
doc[0]=newdoc
print(doc[0])

In [ ]:
print((mat.shape[0]))

In [ ]:
import math
# from textblob import TextBlob as tb

def tf(word, text):
    return text.count(word) / len(text)

# def n_containing(word, bloblist):
#     return sum(1 for blob in bloblist if word in blob.words)

def idf(word, texts):
    count=0
    length=0
    for d in doc:
        length += len(d)
        count += d.count(word)
    return math.log(length) / (1 + count)

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [ ]:
bloblist = doc
deletewords=[]
for i, blob in enumerate(bloblist):
    print("need to delete words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=False)
    for word,score in sorted_words:
        if(round(score,5)<0.00001):
            deletewords.append(word)
            print(deletewords)

#     for word, score in sorted_words[:3]:
#     print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
    

In [ ]:
word=doc[0][1]
count=0
length=0
for d in doc:
    length += len(d)
    count += d.count(word)
print(length,count)

In [ ]:
for word in doc[0]:
    print(doc[0].count(word),word)

In [ ]:
len(doc)